# Exercise 4

Exercise 4 includes a **written assignment** (10 points), a **programming assignment with 2+1 problems** (9 points) and a **feedback/workload assessment assignment** (1 point). For each problem you need to modify the notebook by adding your own programming solutions or written text. Remember to save and commit your changes locally, and push your changes to GitHub after each major change! Regular commits will help you to keep track of your changes (and revert them if needed). Pushing your work to GitHub will ensure that you don't lose any work in case your computer crashes (can happen!).

### Time allocation

**Completing this exercise takes approximately: 15-20 hours** (based on previous year statistics). However, the time it takes can vary significantly from student to student, so we **recommended that you start immediately working on the exercise to avoid issues meeting with the deadline**.

### Due date

You should submit (push) your Exercise answers to your personal Github repository **by Wednesday 28.2**.

### Start your exercise in CSC Notebooks

Before you can start programming, you need to launch the CSC Notebook instance and clone your **personal copy of the Exercise repository** (i.e. something like `exercise-4-htenkanen`) there using Git. If you need help with this, [read the documentation on the course site](https://sustainability-gis.readthedocs.io/en/latest/lessons/L1/git-basics.html).

### Working with Jupyter Notebooks

Jupyter Notebooks are documents that can be used and run inside the JupyterLab programming environment (e.g. at [notebooks.csc.fi](https://notebooks.csc.fi/)) containing the computer code and rich text elements (such as text, figures, tables and links). 

**A couple of hints**:

- You can **execute a cell** by clicking a given cell that you want to run and pressing <kbd>Shift</kbd> + <kbd>Enter</kbd> (or by clicking the "Play" button on top)
- You can **change the cell-type** between `Markdown` (for writing text) and `Code` (for writing/executing code) from the dropdown menu above. 

See [**further details and help from here**](https://pythongis.org/part1/chapter-01/nb/04-using-jupyterlab.html). 
 
### Hints 

If there are general questions arising from this exercise, we will add hints to the course website under [Exercise 4 description](https://sustainability-gis.readthedocs.io/en/latest/lessons/L5/exercise-4.html). 

<hr style="border:2px solid gray">

# Part 1: Written assignment

In the *Sustainable cities and mobilities* and *Mobility analytics* lessons this week, we went through a few ideas related to how mobilities relates to sustainability and how it is possible to analyze observed spatial mobility analytically. 

## Spatial mobility and sustainability (10 points)

Write approx. 0.5-2 pages (A4) of text in English. Remember to **cite your sources appropriately** when you use any literature or other reading materials in your text. 

In this essay, you should cover following questions:
 
- We drive, fly and sail more than ever. What sustainability issues does increased movement of people and objects cause?
- What kind of plausible solutions or improvements exist that can help solving mobility related issues in cities, in between cities, and globally? 
- Do you consider (or have you considered) sustainability related issues regarding your own travel behavior? E.g. in terms of what mode of transport you use for daily travelling, or e.g. regarding the destinations where you travel for holiday?
- When analyzing mobility data (e.g. data obtained from GNSS devices, mobile phone operators, or social media platforms), what challenges there exist related to the nature of the data? Are there any workarounds to overcome these challenges?
 
Use the lesson materials, recommended readings and other literature as a source of information for answering to these.

### Grading criteria for the essay

- Answers to the question(s): 4 points

- Reflection against literature + materials: 4 points

- Fluency / clarity of the text: 1 point

- Appropriate citation practices used: 1 point

----------------

## Answers / reflection

**Add your text here.**

*Hint: To "activate" this cell in Editing mode, double click this cell. If you want to get this cell back in the "Reading-mode", <kbd>Shift</kbd> + <kbd>Enter</kbd>.*

## Hints

- If you need help in Markdown formatting (e.g. how to add headings, bold, italics, links etc.), please take a look at this excellent [guide / cheatsheet](https://www.markdownguide.org/cheat-sheet/) 

<hr style="border:2px solid gray">

# Part 2: Programming assignment

In this exercise, we will practice movement data analysis and visualization. Our overall goal is to learn how to work with spatio-temporal data, and manipulate GPS observations obtained from Helsinki Region Transport showing the movements of public transport vehicles in Helsinki Region. The data was obtained from [HSL GTFS-RT feed](https://hsldevcom.github.io/gtfs_rt/) that provides information about the locations of the vehicles in real time. The frequency of observations in the raw data is 1 second. In this exercise, you will learn how to:

 - filter and resample the data (reduce the number of observations)
 - visualize the movement data with an animation
 - extract specific mobility characteristics from the data (e.g. statistics about average speed)

### Start your exercise in CSC Notebooks

Before you can start programming, you need to launch the CSC Notebook instance and clone your Exercise repository there.
If you need help with this, [read the documentation on the course site](https://sustainability-gis.readthedocs.io/en/latest/lessons/L1/git-basics.html).
 
### Hints 

If there are general questions arising from this exercise, we will add hints to the course website under [Exercise 4 description](https://sustainability-gis.readthedocs.io/en/latest/lessons/L5/exercise-4.html). 

## Helper functions

The following functions are provided for you to make it slightly easier to work through the exercise. Execute the cell below so that you are able to use the functions:

In [ ]:
def downsample_GPS_observations(gdf, group_by_columns, sampling_interval):
    """
    A helper function to downsample the GTFS-RT movement observations.
    
    Parameters
    ----------
    
    gdf : gpd.GeoDataFrame
    
        A GeoDataFrame containing the GPS observations.
        
    group_by_columns : list
    
        A list of columns that will be used to group the observations. 
        Should be a list with column names that represent vehicle_id and direction_id.
        
    sampling_interval : str
    
        Sampling interval provided as text (i.e. a 'Frequency string'). See the available Frequency Strings from a table in the pandas user guide:
        https://pandas.pydata.org/docs/user_guide/timeseries.html#dateoffset-objects
    
    """

    # Container for resampled GeoDataFrames
    sampled =  []

    # Resample the data
    grouped = selected.groupby(group_by_columns)
    
    # The group by columns should include vehicle_id and direction_id
    for (vehicle_id, direction_id), group in grouped:
        # Resample the group and take one observation per 10 seconds
        sample = group.resample(sampling_interval).first()
        sampled.append(sample)

    # Merge all samples
    sampled = pd.concat(sampled)  
    sampled = sampled.dropna(subset=["geometry"])
    
    return sampled


def trajectory_points_to_linestring(trajectory, speed_column="speed_kmph"):
    """
    A helper function to create a GeoDataFrame with LineString geometry based on the points of a given trajectory.
    Also calculates the average and standard deviation of the speed of the trajectory.
    
    Parameters
    ----------
    
    trajectory : mpd.Trajectory
    
        A MovingPandas Trajectory object which will be used for creating the GeoDataFrame with LineString geometry 
        and some relevant trajectory-related attributes.
        
    speed_column : str
    
        A column containing the travel speed.
    
    """
    # Parse the GeoDataFrame
    df = trajectory.df
    
    # Remove observations that are very slow (< 1 kmph)
    df = df.loc[df[speed_column] > 1].copy()
    
    # If there were only less than 3 observations return empty GeoDataFrame
    # This will clean the data from trajectories 
    if len(df) < 3:
        return None
    
    # Sort the observations
    df = df.sort_index()
    
    # What is the average speed of the route
    avg_speed = df[speed_column].mean()
    std_speed = df[speed_column].std()
    
    # Parse other useful values
    vehicle_id = df["vehicle_id"].unique()[0]
    route_id = df["route_id"].unique()[0]
    direction_id = df["direction_id"].unique()[0]
    start_time = df["timestamp"].head(1).values[0]
    
    # Create LineString geometry
    geom = trajectory.to_linestring()
    
    # Store the relevant data
    data = {"geometry": [geom], "route_id": route_id, 
            "vehicle_id": vehicle_id, 
            "direction_id": direction_id,
            "avg_speed": avg_speed,
            "std_speed": std_speed,
           }
    return gpd.GeoDataFrame(data, crs="epsg:4326")

## Problem 1 - Prepare and filter data (5 points)

In this problem, the objective is to prepare and clean the GPS data. 

**Task 1.1.** Read the GPS data, create geometry column based on coordinates and create a DateTimeIndex as follows:

1. Read the GPS observations from the Zipped CSV file `helsinki_gtfs_rt.zip` into pandas `DataFrame` using pandas function `pd.read_csv()`. The data is available from the `data` directory. Store the data into variable called `data`. 
   - Notice that [you can read the data directly from the zipfile](https://stackoverflow.com/a/32993553) without needing to extract it.
   - Hint: In case you are not familiar with `.read_csv()` function, [read more from here](https://pythongis.org/part1/chapter-03/nb/00-pandas-basics.html#reading-tabular-data).
    
2. Create a new `"geometry"` column with Point geometries based on `lat` and `lon` columns. 
   - For doing this, you can take advantage of `gpd.points_from_xy()` function available in geopandas (see [details from the docs](https://geopandas.org/en/stable/docs/reference/api/geopandas.points_from_xy.html)). 
   - Remember that `lon` represents `x` coordinates and `lat` stands for `y` coordinates.
   
3. Create a `GeoDataFrame` based on the `data` and use the `WGS84` as the coordinate reference system (CRS).
   - In case you are not familiar how to create a `GeoDataFrame` from pandas `DataFrame`, take a look at [this code snippet](https://gis.stackexchange.com/a/174168).
   - In case you do not know how to specify WGS84 coordinate system, check [this documentation](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.set_crs.html).
   - Hint: You want to specify the CRS information as a EPSG code. Google what is the EPSG code for WGS84.
4. Create a DateTime index for the GeoDataFrame based on the `timestamp` column.
   - You can use the `pd.to_datetime()` function to create a `datetime` column and specify this column as index using `.set_index()` function.
   - Hint: Take a look at the [tutorial 4](https://sustainability-gis.readthedocs.io/en/latest/lessons/L5/mobility-analytics.html#constructing-trajectories) from our course pages.
   - In case you are interested to learn more about temporal data processing in pandas, take a look at materials from [pythongis.org website](https://pythongis.org/part1/chapter-03/nb/03-temporal-data.html).
 

<br>

Please write your solution to the cell below (remove the `raise NotImplementedError()` code). You can create new cells as well if needed.

In [ ]:
# Filepath to data
fp = "data/helsinki_gtfs_rt.zip"
 
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
raise NotImplementedError()

**Task 1.2.** Answer to following questions (use programming to find the answers):

- When was the first observation recorded?
- When was the last observation recorded?
- On which weekday has the data been recorded? (e.g. Monday, Tuesday etc.)

_**Hints:**_
- Your `GeoDataFrame` should at this point contain a `DateTimeIndex`. Use this data to find the answers. 
- You can explore the properties of the `DateTimeIndex` by using standard pandas methods, such as `.min()`, `.max()`. You can read more details from the [pythongis.org](https://pythongis.org/part1/chapter-03/nb/03-temporal-data.html#selecting-data-based-on-datetimeindex) website.


<br>

Please write your solution to the cell below (remove the `raise NotImplementedError()` code). You can create new cells as well if needed.

In [ ]:
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
raise NotImplementedError()

**Task 1.3.** Explore the temporal profile of the data by making an interactive histogram from the GPS data that shows the number of observations per minute:

1. Create a new column called `"timestamp_dt"` based on the `"timestamp"` column in a similar manner as we did in Task 1.1. 

2. Resample the observations into 1 minute intervals based on the `"timestamp_dt"` and plot the temporal profile:

   - You can use the `.resample("1T")` function in pandas to resample the observations into 1 minute intervals.
   - You can count the number of observations per minute by *chaining* the `.count()` method after the previous `.resample("1T")` call. 
      - If you do not know what chaining means, read the section "Method Chaining Using Pandas" from [this documentation](https://www.tutorialspoint.com/Explain-Python-class-method-chaining).
   - Finally you can chain the plotting function `.hvplot()` at the end of the previous two commands.

   - _**Hint:**_ Take a look at the Exercise 4 hints that can be found [from our course website](https://sustainability-gis.readthedocs.io/en/latest/lessons/L5/exercise-4.html#how-to-count-values-in-pandas-and-visualize-them-interactively) which shows how to do similar things (but on a monthly level).

As a result, you should get something like this:

![Temporal profile before filteaaar.](img/Temporal_profile_before_filter.png)

<br>

Please write your solution to the cell below (remove the `raise NotImplementedError()` code). You can create new cells as well if needed.


In [ ]:
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
raise NotImplementedError()

**Task 1.4.** Based on the information from the interactive histogram, find out the answers to following questions (by manually exploring the interactive visualization):
 
1. when was the first moment in time (minute) when there were more than 4 thousand observations per minute? 
   - store the answer to a variable `start_time` in a following format `"YYYY-MM-DD HH:MM"`
2. when was the last moment in time (minute) when there were more than 4 thousand observations per minute?
   - store the answer to a variable `end_time` in a following format `"YYYY-MM-DD HH:MM"`
   
<br>

Please write your solution to the cell below (remove the `raise NotImplementedError()` code). You can create new cells as well if needed.

In [ ]:
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
raise NotImplementedError()

**Task 1.5**. Based on the start and end time that you found out in the previous step:
1. Select the observations that are between the given moments in time and store the results in variable called `selected`. 
   - You can select the observations by using the `.loc[start_time: end_time]`.
   - In case you want to read more details about slicing a DataFrame based on time, read this section from the [pythongis.org](https://pythongis.org/part1/chapter-03/nb/03-temporal-data.html#selecting-data-based-on-datetimeindex).
2. Visualize the `selected` data with a histogram using the combination of `.resample()`, `.count()` and the `.hvplot()` in a similar manner as in Task 1.3. 

As a result after selecting the data, you should have something like the plot below where the number of observations per minute never drops below 4000 (as visible from the y-axis):
 
![Temporal profile after filter](img/Temporal_profile_after_filter.png)

<br>

Please write your solution to the cell below (remove the `raise NotImplementedError()` code). You can create new cells as well if needed.

In [ ]:
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
raise NotImplementedError()

## Problem 2 - Create an animation based on the movements (4 points)

In this problem, the objective is to **i)** reduce the number of observations by downsampling our data (i.e. only keeping a record every 10 seconds) and **ii)** create an animation out of our data. 

**Task 2.1.** To downsample the data, you should:

1. First downsample the GPS observations stored in the `selected` GeoDataFrame which you created in previous step in **Task 1.5**: 
   - For doing this, use the `downsample_GPS_observations()` function provided to you in the **Helper functions**. Read the function description to understand how it works and what parameters you need to use.
   - You should downsample the data in a way that one observation per every 10 seconds is kept. You can specify the sampling rate with `sampling_interval` parameter.
   - The sampling interval is passed as a text following the so called "Frequency String" format as described in the `DateOffsets` table in [pandas documentation which you can read from here](https://pandas.pydata.org/docs/user_guide/timeseries.html#dateoffset-objects).
   - Downsampling is based on groups of GPS observations which are selected from the data based on `"vehicle_id"` and `"direction_id"`. This means that the function will split the data where individual trajectory for a given vehicle and to given direction will be downsampled (such a metro from Tapiola to Mellunmäki or vice versa). You can use the `grouping_columns` variable (a list) and pass it to the `group_by_columns` parameter. 
   - Store the results in a variable called `downsampled`. This GeoDataFrame should contain the same attributes as the `selected` GeoDataFrame, but much fewer records (as only 1 record per 10 seconds is kept).

2. Make a copy of the `downsampled` GeoDataFrame and store it into a new variable `visualization_data`.
3. Reset the index of the `visualization_data` GeoDataFrame.
4. Keep only following columns in the `visualization_data` GeoDataFrame: `"geometry", "speed", "timestamp", "direction_id", "vehicle_id", "route_id"`
    - You can select specific columns from a GeoDataFrame by passing a list of columns inside square brackets, such as `gdf[ ["column1", "column2", "etc"] ]`
    - Store the results of this selection into a variable `visualization_data` (i.e. we are overwriting the previous content of this variable)

<br>

Please write your solution to the cell below (remove the `raise NotImplementedError()` code). You can create new cells as well if needed.

In [ ]:
# The columns which should be used to group the observations
grouping_columns = ["vehicle_id", "direction_id"]

# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
raise NotImplementedError()

**Task 2.2.** Create an animation based on the movement data.

1. Create first an empty `KeplerGl()` map as shown in the [last section of Tutorial 4](https://sustainability-gis.readthedocs.io/en/latest/lessons/L5/mobility-analytics.html#animating-and-exploring-the-trajectory-characteristics-in-3d) in our course website.
2. Add the `visualization_data` from previous step to KeplerGl using the `.add_data()` function and store it as a layer called `"trajectories"` in a similar manner as shown in [Tutorial 4](https://sustainability-gis.readthedocs.io/en/latest/lessons/L5/mobility-analytics.html#animating-and-exploring-the-trajectory-characteristics-in-3d).
3. Using the `KeplerGl` interface, create an animation in which:
   - the color of the points is based on `speed` attribute
   - the `timestamp` attribute is used as a filter which allows you to select the data based on a desired timewindow and animate the movements.
   
**Hint:** Take a look at the [Tutorial video from the course site](https://aalto.cloud.panopto.eu/Panopto/Pages/Viewer.aspx?id=a7552d81-df57-45e1-afb3-af990091b126), which shows the basic functionalities of KeplerGl and how to animate the ship observations from the Tutorial 4. Applying these same tricks you can create an animation from the public transport vehicle movements.

The output animation could look something like following in which the speed is used as source for coloring the points

![PT Movements](img/GTFS_RT_Movements.gif)

Please write your solution to the cell below (remove the `raise NotImplementedError()` code). You can create new cells as well if needed.

In [ ]:
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
raise NotImplementedError()

**Task 2.3.** Save the animation into a HTML file and add it to version control. **After you have created the animation**: 

- Save the map and store it into a HTML file called `HSL_trajectories.html`. 
  - Hint: Take a look at the Tutorial 4 to find out how to do this.
- **Add the `HSL_trajectories.html` to Git and commit the file**. In this way, the resulting visualization will also be visible in Github.
  - When you open the `Git` tab in JupyterLab, you should see the `HSL_trajectories.html` in the `Untracked` section of the Git panel.
  - Click (select) the `HSL_trajectories.html` file from the Untracked list and press the `+` button on the right. This will add the file to staging area, and you can make a commit in a similar manner as before when working on the exercises.
   
<br>

Please write your solution to the cell below (remove the `raise NotImplementedError()` code). You can create new cells as well if needed.

In [ ]:
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
raise NotImplementedError()

## Problem 3 (optional): Extracting mobility characteristics using trajectory data mining 

*This is an optional problem for those of you who want to dive a bit deeper into trajectory data mining. This problem does not have any "official" points, but if you advance with this problem, the work on this one may grant you some extra points. However, the maximum points from this Programming Assignment is still 9 points.*

In this problem, the objective is to calculate some basic movement characteristics based on our downsampled data in the `downsampled` GeoDataFrame (as was introduced in tutorial 4). 

**Task 3.1**. Calculate the mobility characteristics in a following manner:

1. Create a new column called `speed_kmph` to the `downsampled` GeoDataFrame into which you should calculate the speed as kilometers per hour.
   - Use the column `speed` as the input for this calculation which contains the speed information as a **meters per second**. 
   - You need to convert the speed in meters per second to kilometers per hour by doing a simple math calculation with pandas (you should figure out the formula yourself).

2. Ensure that the CRS of the `downsampled` is WGS84:
   - Use the `.set_crs()` to specify the Coordinate Reference system using the EPSG code for WGS84 in case the GeoDataFrame does not contain valid CRS information. 

3. Group the sampled GeoDataFrame (from Problem 2 having 10 second intervals) based on `direction_id` and `vehicle_id`.
   - Hint: You can take a look at how things are done inside the `downsample_GPS_observations()` function.
   
4. Iterate over the groups and on each iteration run the following steps (*Hint: iterate first only one group, so that you don't waste time going through all the time while writing the functionalities*):

   1. Create a trajectory collection out of the group if the maximum speed of the observations is larger than 0 (reported as meters per second)
       - Hint: Check if the maximum value of the column `speed` is zero. If it is, do not proceed further and continue to the next iteration. In case it is not -> Create a `TrajectoryCollection` from the group using `vehicle_id` as the identifier in a somewhat similar way as shown in the [Tutorial 4](https://sustainability-gis.readthedocs.io/en/latest/lessons/L5/mobility-analytics.html#constructing-trajectories).
   2. Add speed to the TrajectoryCollection (overwrite existing one) following an example from Tutorial 4.
   3. If there aren't any trajectories in our TrajectoryCollection, continue to the next one by using Python statement `continue` (not familiar with continue statement? [Read more here](https://www.w3schools.com/python/ref_keyword_continue.asp)).
       - Hint: Check the length of the `my_trajectory_collection_from_previous_step.trajectories`. If the length is zero, continue to the next group using the `continue` statement.
   4. In case there was a valid trajectory: Split the given trajectory based on criteria that if there is a 5-minute time gap between observations, the trajectory should be splitted into multiple ones. 
      - Use the `mpd.ObservationGapSplitter()` functionality to do this (find information from the [movingpandas docs](https://movingpandas.readthedocs.io/en/main/trajectorysplitter.html)).
      - Store the result of the splitting operation into a variable called `splitted`.
      - The result from this step will be a new movingpandas `TrajectoryCollection`.
   5. Iterate over the trajectories in the `splitted` TrajectoryCollection and use the function `trajectory_points_to_linestring()` provided for you in the Helper functions to convert the individual trajectory into a GeoDataFrame. Store the result returned by the function into a variable `mobility_characteristics` and append this GeoDataFrame into the list `results` provided to you as the first line in the cell below. 
      - This GeoDataFrame containing LineString geometry of the trajectory and some basic mobility characteristics about the trajectory, such as average speed. 
      - Take a look inside the `trajectory_points_to_linestring()` function if you want to understand in more detail how the characteristics are calculated. 

5. After you have iterated all the trajectories and calculated the mobility characteristics, combine all the GeoDataFrames in the `results` list by using the `pd.concat()` function.
   - Store the combined results into a variable `results` (i.e. overwrite the previous contents).
   - In case you are not familiar with `pd.concat()`, read the [pandas documentation](https://pandas.pydata.org/docs/reference/api/pandas.concat.html).
   

<br>

Please write your solution to the cell below (remove the `raise NotImplementedError()` code). You can create new cells as well if needed.

In [ ]:
# Container for the resulting trajectory GeoDataFrames
results = []

# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
raise NotImplementedError()

**Task 3.2.** Calculate some simple mobility statistics:

- Based on the trajectory data, answer to following questions using programming:
  - What is the average speed of trajectories in our data (considering all trajectories)?
  - What is the maximum speed of trajectories in our data (considering all trajectories)?
  - What is the total length of all trajectories in our data? (in kilometers)
  
<br>

Please write your solution to the cell below (remove the `raise NotImplementedError()` code). You can create new cells as well if needed.

In [ ]:
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
raise NotImplementedError()

**Task 3.3.** Plot the trajectories.

1. Visualize the trajectories by creating an interactive map. 
   - Use the `avg_speed` attribute as a source column for coloring the trajectories.
   
2. Based on your visualization, which transport lines seem to have the highest average speed?
   - **Add your answer here:**: 
   
<br>

Please write your solution to the cell below (remove the `raise NotImplementedError()` code). You can create new cells as well if needed.

In [ ]:
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
raise NotImplementedError()

**Task 3.4**.  Create an interactive movement profile (a histogram) based on the average speed that shows the distribution of trajectory speeds in the data.

- You can use the `hvplot.hist()` function to make the visualization.
   - Hint: Take a look at the [Exercise 4 hints](https://sustainability-gis.readthedocs.io/en/latest/lessons/L5/exercise-4.html#how-to-plot-an-interactive-histogram) to see an example how you can create an interactive histogram using `hvplot` library.
   
As a result, you should have something like below:

![Average speeds profile](img/Average_speeds.png)

<br>

Please write your solution to the cell below (remove the `raise NotImplementedError()` code). You can create new cells as well if needed.

In [ ]:
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
raise NotImplementedError()

## Problem 4 - How long did it take? Optional feedback (1 point)

To help developing the exercises, and understanding the time that it took for you to finish the Exercise, please provide an estimate of how many hours you spent for doing this exercise? *__Hint:__ To "activate" this cell in Editing mode, double click this cell. If you want to get this cell back in the "Reading-mode", press <kbd>Shift</kbd> + <kbd>Enter</kbd>.*


I spent approximately this many hours: **X hours**

In addition, if you would like to give any feedback about the exercise (optional), please provide it below:

**My feedback:**